In [66]:
import pandas as pd
import numpy as np

df = pd.read_csv('poetry.csv')
df = df.drop(columns=df.columns[[5, 6, 7, 8]], axis=1)

df = df[:512]

def get_century_roman(year):
	century = year // 100 + 1
	roman_numerals = {
		1: 'I', 2: 'II', 3: 'III', 4: 'IV', 5: 'V',
		6: 'VI', 7: 'VII', 8: 'VIII', 9: 'IX', 10: 'X',
		11: 'XI', 12: 'XII', 13: 'XIII', 14: 'XIV', 15: 'XV',
		16: 'XVI', 17: 'XVII', 18: 'XVIII', 19: 'XIX', 20: 'XX', 21: 'XXI'
	}
	return roman_numerals[century]

def proc_row(row):
	name = row['name'] if not pd.isna(row['name']) else row['text'].split('\n')[0]
	author = row['author'] if not pd.isna(row['author']) else 'неизвестен'

	date_from, date_to = row['date_from'], row['date_to']
	if pd.isna(date_from) and not pd.isna(date_to): date_from = date_to
	if pd.isna(date_to) and not pd.isna(date_from): date_to = date_from
	
	century = ''
	epoch = ''
	if pd.isna(date_to) and pd.isna(date_from): 
		date_to = date_from = -1
	else:
		century = f'{get_century_roman(date_to)} ({int(date_to//100+1)})'
		if date_to < 1500: epoch = 'Древнерусская поэзия'
		if 1750 <= date_to < 1850: epoch = 'Золотой век'
		if 1850 <= date_to < 1900: epoch = 'Реализм и натурализм'
		if 1900 <= date_to < 1950: epoch = 'Серебряный век'
		#if 1950 <= date_to < 1985: epoch = 'Советская поэзия'
		if 1990 <= date_to < 2010: epoch = 'Бронзовый век'
		if 2010 <= date_to < 2100: epoch = 'Современная поэзия'
	
	#text = '\n'.join(row['text'].split('\n')[:4])
	text = row['text'].replace('\r', '')

	line_count = text.count('\n')
	word_count = text.count(' ') + line_count

	return pd.Series({'author': author, 'name': name, 'text': text, 
				   	  'date_from': int(date_from), 'date_to': int(date_to), 'century': century, 'epoch': epoch, 
				  	  'line_count': line_count, 'word_count': word_count})

df = df.apply(proc_row, axis=1)

df.head()

,author,name,text,date_from,date_to,century,epoch,line_count,word_count
0,Михаил Лермонтов,Забывши волнения жизни мятежной...,"Забывши волнения жизни мятежной,\nОдин жил в п...",1829,1829,XIX (19),Золотой век,7,29
1,Сергей Есенин,"Нивы сжаты, рощи голы...","Нивы сжаты, рощи голы,\nОт воды туман и сырост...",1917,1918,XX (20),Серебряный век,11,46
2,Игорь Северянин,ЧАРЫ ЛЮЧИНЬ,Лючинь печальная читала вечером ручьисто-вкрад...,1919,1919,XX (20),Серебряный век,11,85
3,Анатолий Жигулин,Золото,"Глыбу кварца разбили молотом,\nИ, веселым огне...",1963,1963,XX (20),,29,122
4,Николай Тихонов,Хоровод в Сульдуси,"Хлынул дождь, когда девушки, встав в хоровод,\...",1937,1940,XX (20),Серебряный век,15,92


In [67]:
len(df)

512

In [68]:
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
tqdm.pandas()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

emb_model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
#emb_model = SentenceTransformer('intfloat/multilingual-e5-large')
emb_model.to(device)

def emb_row(row):
	metadata = f'название: {row["name"]}, '
	if row['author'] != 'неизвестен': metadata += f'автор: {row["author"]}, '

	if row['date_from'] != -1:
		metadata += f'период написания: {row["century"]} век, {row["date_from"]} - {row["date_to"]} год, '

	if len(row['epoch']) > 0: metadata += row['epoch'] + ', '

	metadata += f'объем: количество строк {row["line_count"]}, '
	metadata += f'количество четверостиший: {int(row["line_count"]//4)}, '
	metadata += f'количество слов: {row["word_count"]}'
	
	row['text_embedding'] = emb_model.encode(row['text'])
	row['metadata_embedding'] = emb_model.encode(metadata)

	return pd.Series(row)


df = df.progress_apply(emb_row, axis=1)


Using device: cpu


100%|██████████| 512/512 [01:33<00:00,  5.46it/s]


In [69]:
df.head()
#df.to_csv('poetry_data_prepared.csv')

,author,name,text,date_from,date_to,century,epoch,line_count,word_count,text_embedding,metadata_embedding
0,Михаил Лермонтов,Забывши волнения жизни мятежной...,"Забывши волнения жизни мятежной,\nОдин жил в п...",1829,1829,XIX (19),Золотой век,7,29,"[-0.009159689, 0.020552872, -0.02544283, -0.01...","[0.07024827, 0.004548956, -0.050651815, -0.028..."
1,Сергей Есенин,"Нивы сжаты, рощи голы...","Нивы сжаты, рощи голы,\nОт воды туман и сырост...",1917,1918,XX (20),Серебряный век,11,46,"[-0.019855766, 0.0011318808, 0.026735159, -0.0...","[0.023410179, 0.030621158, -0.013930113, -0.03..."
2,Игорь Северянин,ЧАРЫ ЛЮЧИНЬ,Лючинь печальная читала вечером ручьисто-вкрад...,1919,1919,XX (20),Серебряный век,11,85,"[-0.052177265, -0.006428905, 0.024191873, -0.0...","[0.035727117, 0.040428042, -0.0011023306, -0.0..."
3,Анатолий Жигулин,Золото,"Глыбу кварца разбили молотом,\nИ, веселым огне...",1963,1963,XX (20),,29,122,"[-0.029019197, -0.010935792, 0.0038938446, -0....","[-0.0013262919, 0.024923684, 0.0114643015, -0...."
4,Николай Тихонов,Хоровод в Сульдуси,"Хлынул дождь, когда девушки, встав в хоровод,\...",1937,1940,XX (20),Серебряный век,15,92,"[0.026022725, 0.021309432, -0.027745932, 0.018...","[-0.01996481, 0.04875582, 0.0017633839, -0.049..."


In [75]:
import numpy as np
from tqdm import tqdm

def cosine_similarity(a, b):
	return np.dot(a, b) / np.linalg.norm(a) / np.linalg.norm(b)

def find_poem(request, top_n=10, lim=0.3):
	req_emb = emb_model.encode(request)

	texts_datas = []

	for index, row in tqdm(df.iterrows()):
		text_sim = cosine_similarity(req_emb, row['text_embedding'])
		req_sim = cosine_similarity(req_emb, row['metadata_embedding'])
		score = text_sim + req_sim

		if score >= lim:
			texts_datas.append({'poem': row['text'], 'metadata': {k: v for k, v in row.items() if k != 'text' and k != 'metadata_embedding' and k != 'text_embedding'}, 
					   			'score': score, 'text_sim': text_sim, 'req_sim': req_sim})
	
	texts_datas.sort(key=lambda x: x['score'])
	return texts_datas[-top_n:]


#request = 'из окны видны грустные пожелтевшие деревья но все же это моя любимая романтика'
#request = 'научный энтузиазм и борение за оригинальную идею'
#request = 'поднять! сделать! решительность! я! восклецания, Владимир Маяковский'
#request = 'дыр бул щыл убеш щур скум вы со бу р л эз'
request = 'стихотворение, поэзия, поэт, творчество, искусство, серебрянный век, Автор: Владимир Маяковский'
#request = 'Автор: Александр Крученых Автор: Александр Крученых Автор: Александр Крученых Автор: Александр Крученых Автор: Александр Крученых'

finded = find_poem(request)

for d in finded:
	print(d['poem'])
	print({k: v for k, v in d.items() if k != 'poem'})
	print('------')

512it [00:00, 6667.24it/s]

Так просто можно жизнь покинуть эту,
Бездумно и безбольно догореть.
Но не дано Российскому поэту
Такою светлой смертью умереть.
Всего верней свинец душе крылатой
Небесные откроет рубежи,
Иль хриплый ужас лапою косматой
Из сердца, как из губки, выжмет жизнь.
{'metadata': {'author': 'Анна Ахматова', 'name': 'Памяти Сергея Есенина', 'date_from': 1925, 'date_to': 1925, 'century': 'XX (20)', 'epoch': 'Серебряный век', 'line_count': 7, 'word_count': 38}, 'score': 0.6133378, 'text_sim': 0.3088695, 'req_sim': 0.3044683}
------
Нет.
Это неправда.
Нет!
И ты?
Любимая,
за что,
за что же?!
Хорошо -
я ходил,
я дарил цветы,
я ж из ящика не выкрал серебряных ложек!
Белый,
сшатался с пятого этажа.
Ветер щеки ожег.
Улица клубилась, визжа и ржа.
Похотливо взлазил рожок на рожок.
Вознес над суетой столичной одури
строгое -
древних икон -
чело.
На теле твоем - как на смертном одре -
сердце
дни
кончило.
В грубом убийстве не пачкала рук ты.
Ты
уронила только:
"В мягкой постели
он,
фрукты,
вино на ладони ночн